In [1]:
import math
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Friction coefficients
friction_data = {
    'Asphalt': {'Dry': 0.75, 'Wet': 0.55, 'Icy': 0.15},
    'Highway': {'Dry': 0.80, 'Wet': 0.60, 'Icy': 0.10},
    'Gravel': {'Dry': 0.50, 'Wet': 0.30, 'Icy': 0.05}
}

g = 9.81  # gravity

# Animation function
def animate_braking(speed_kmh, road, cond, max_stop_dist):
    μ = friction_data[road][cond]
    v0 = speed_kmh / 3.6  # convert to m/s
    braking_distance = v0**2 / (2 * μ * g)

    # Alert if overshoot
    if braking_distance > max_stop_dist:
        display(HTML(f"<b style='color:red;'>⚠️ Warning:</b> The car will overshoot the maximum safe stopping distance!<br>Braking Distance = <b>{braking_distance:.2f} m</b>, Max Allowed = <b>{max_stop_dist:.2f} m</b>"))
    else:
        display(HTML(f"<b style='color:green;'>✅ Safe:</b> Braking distance = <b>{braking_distance:.2f} m</b>"))

    # Generate animation frames
    step = max(1, int(braking_distance)//80)  # smaller step = smoother & faster
    positions = list(range(0, int(braking_distance)+1, step))

    frames = [
        go.Frame(
            data=[go.Scatter(x=[pos], y=[0], mode='markers', marker=dict(size=20, color='blue'))],
            name=str(pos)
        ) for pos in positions
    ]

    fig = go.Figure(
        data=[go.Scatter(x=[0], y=[0], mode='markers', marker=dict(size=20, color='blue'))],
        layout=go.Layout(
            title=f"🚗 Car Braking – Speed: {speed_kmh} km/h, {road}, {cond}",
            xaxis=dict(title='Distance (m)', range=[0, max(braking_distance, max_stop_dist) + 10]),
            yaxis=dict(visible=False),
            shapes=[
                dict(type='line', x0=max_stop_dist, x1=max_stop_dist, y0=-1, y1=1, line=dict(color='red', dash='dash'))
            ],
            updatemenus=[dict(
                type="buttons",
                buttons=[dict(label="▶ Play", method="animate", args=[None, {"frame": {"duration": 30}, "fromcurrent": True}])]
            )]
        ),
        frames=frames
    )

    fig.show()

# Widgets
speed_input = widgets.IntText(value=60, description='Speed (km/h):')
road_dropdown = widgets.Dropdown(options=list(friction_data.keys()), value='Asphalt', description='Road:')
condition_dropdown = widgets.Dropdown(options=['Dry', 'Wet', 'Icy'], value='Dry', description='Condition:')
max_dist_input = widgets.FloatText(value=40, description='Max Stop Dist:')
run_button = widgets.Button(description="Run Animation", button_style='success')

def on_run_clicked(b):
    clear_output(wait=True)
    display(speed_input, road_dropdown, condition_dropdown, max_dist_input, run_button)
    try:
        animate_braking(
            speed_kmh=speed_input.value,
            road=road_dropdown.value,
            cond=condition_dropdown.value,
            max_stop_dist=max_dist_input.value
        )
    except Exception as e:
        print(f"Error: {e}")

run_button.on_click(on_run_clicked)

display(speed_input, road_dropdown, condition_dropdown, max_dist_input, run_button)

IntText(value=60, description='Speed (km/h):')

Dropdown(description='Road:', options=('Asphalt', 'Highway', 'Gravel'), value='Asphalt')

Dropdown(description='Condition:', options=('Dry', 'Wet', 'Icy'), value='Dry')

FloatText(value=40.0, description='Max Stop Dist:')

Button(button_style='success', description='Run Animation', style=ButtonStyle())